## importing libraries

In [2]:
# !python -m spacy download en_core_web_sm


In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 18500000

In [5]:
import pandas as pd
df = pd.read_csv("sample_text.csv", encoding="latin1")

In [7]:
df.head()

,Response
0,Safety has been a facade. Education for educat...
1,More time should be spent with education aroun...
2,I am a special education teacher with autistic...
3,It has been a very challenging year and I pers...
4,I support the decision to keep students in sch...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Response  6 non-null      object
dtypes: object(1)
memory usage: 180.0+ bytes


In [10]:
df.describe()

,Response
count,6
unique,6
top,Safety has been a facade. Education for educat...
freq,1


The first step is to join all the responses into a single mega string, since I want to analyze the responses as a whole. For this, I use pandas’ handy cat string method to concatenate all the strings in the Reponse column.

In [6]:
all_text = df.Response.str.cat(sep = ' ')

Now create a spaCy document with that text. I don’t need the named entity recognizer (NER) so I disable that to save on memory and computing time.

In [11]:
doc = nlp(all_text, disable = ['ner'])